In [75]:
import pandas as pd

df = pd.read_csv('TCS.NS_Day_Wise.csv')

df['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')

df['Week_Number'] = df['Date'].dt.strftime('%Y%U').astype('int')

df = df.sort_values(by=['Date'], ascending=True)

df[df['Week_Number'] == 200232]

,Date,Open,High,Low,Close,Adj Close,Volume,Week_Number
0,2002-08-12,38.724998,40.000000,38.724998,39.700001,28.128597,212976,200232
1,2002-08-13,39.750000,40.387501,38.875000,39.162498,27.747759,153576,200232
2,2002-08-14,39.250000,39.250000,35.724998,36.462502,25.834736,822776,200232
3,2002-08-15,36.462502,36.462502,36.462502,36.462502,25.834736,0,200232
4,2002-08-16,36.275002,38.000000,35.750000,36.375000,25.772736,811856,200232


In [76]:
# Week level Agg

tf=df

# Week's Highestm Lowest trading price and Total Vol traded
mf = tf.groupby('Week_Number').aggregate({'High':['max'],'Low':['min'],'Volume':['sum']}).reset_index()
mf.columns = ['Week_Number','Max_High','Min_Low','Total_Volume']

# Week's open and close price
wfh = tf.loc[:,['Week_Number','Open']].groupby('Week_Number').head(1)
wft = tf.loc[:,['Week_Number','Close']].groupby('Week_Number').tail(1)

# Join all the Agg datasets 
wf = pd.merge(wfh,wft,how='left',left_on='Week_Number',right_on='Week_Number')
tf = pd.merge(mf,wf,how='left',left_on='Week_Number',right_on='Week_Number')

tf = tf.rename(columns={'Close':'End_Price','Week_Number':'row'})

tf

,row,Max_High,Min_Low,Total_Volume,Open,End_Price
0,200232,40.387501,35.724998,2001184,38.724998,36.375000
1,200233,43.625000,35.137501,18564720,36.674999,42.299999
2,200234,45.125000,40.287498,12945024,42.974998,44.049999
3,200235,45.974998,41.012501,6795040,44.750000,41.474998
4,200236,43.224998,39.500000,2637016,39.500000,40.987499
...,...,...,...,...,...,...
1113,202339,3620.000000,3505.550049,8505518,3602.500000,3528.600098
1114,202340,3634.949951,3477.300049,8802745,3534.199951,3621.399902
1115,202341,3679.000000,3525.000000,15639277,3645.050049,3570.850098
1116,202342,3574.649902,3444.050049,7747170,3566.000000,3494.550049


In [77]:
# Func to find support and resistance points
def band(df,i):
  support = df.iloc[i-1,1] >= df.iloc[i,1] <= df.iloc[i+1,1] and df.iloc[i-2,1] >= df.iloc[i,1] <= df.iloc[i+2,1]
  resistance = df.iloc[i-1,1] <= df.iloc[i,1] >= df.iloc[i+1,1] and df.iloc[i-2,1] <= df.iloc[i,1] >= df.iloc[i+2,1]
  if support:
    return 'S'
  elif resistance:
    return 'R'
  else:
    return 'N' 


wf = tf

# Add Flag to mark Support and Resistance
wf['Flag'] = ''


# Find Support and Resistance Points
for index in range(2,len(wf)-2):
        wf.iloc[index,-1] = band(wf,index)


wf.dtypes

row               int64
Max_High        float64
Min_Low         float64
Total_Volume      int64
Open            float64
End_Price       float64
Flag             object
dtype: object

In [78]:
# Trend Line Calculation

def band_cal(flag,rsf):
    resistance_df = rsf[ rsf['Flag'] == flag ]
    #print(resistance_df)

    resistance_df = resistance_df.loc[:,['row','End_Price']]

    pts_gph,nt_pts_gph = {},{}
    ind = {}

    for index,row in resistance_df.iterrows():
        pts_gph[row['End_Price']] = {}
        nt_pts_gph[row['End_Price']] = {}

        for i,r in resistance_df[resistance_df['row'] != row['row']].iterrows():
            slope = (r['End_Price'] - row['End_Price']) / (r['row'] - row['row'])
            
            if -1 <= slope <= 1:
                try:
                    pts_gph[row['End_Price']][str(slope)].append(r['End_Price'])
                except:
                    pts_gph[row['End_Price']][str(slope)] = [row['End_Price']]
                    pts_gph[row['End_Price']][str(slope)].append(r['End_Price'])

            else:
                try:
                    nt_pts_gph[row['End_Price']][str(slope)].append(r['End_Price'])
                except:
                    nt_pts_gph[row['End_Price']][str(slope)] = [row['End_Price']]
                    nt_pts_gph[row['End_Price']][str(slope)].append(r['End_Price'])


    #print(pts_gph)
    #print(nt_pts_gph)

    pt,sl,sp = 0,[],0
    pts_gph_keys = list(pts_gph.keys())
    for j in range(1,len(pts_gph_keys)+1):
        i = pts_gph_keys[-j]
        for s in pts_gph[i]:
            if len(pts_gph[i][s]) > pt:
                pt = len(pts_gph[i][s])
                sl = pts_gph[i][s]
                sp = [i,s]


    sr = resistance_df[resistance_df['End_Price'] == sp[0]].row.values[0].item()

    c = float(sp[0].item()) - ( float(sp[1]) * int(sr) )

    return float(sp[1]),c

In [79]:
print(f"Resistance Line = {band_cal('R',wf)}")
print(f"Support Line = {band_cal('S',wf)}")

Resistance Line = (0.8798613371672453, -174461.17272610133)
Support Line = (-0.0, 38.38750076293945)
